In [1]:
import numpy as np
import pandas as pd
import re
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV,StratifiedShuffleSplit
from sklearn.metrics import   roc_auc_score,multilabel_confusion_matrix
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.feature_selection import SelectKBest,mutual_info_classif,f_classif
    
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords

submission_file = pd.read_csv( '../.data/sample_submission.csv')

Xtrain = pd.read_csv('../.data/train_new_features.csv',index_col='id')
Xtest = pd.read_csv('../.data/test_new_features.csv',index_col='id')

Xtr = Xtrain[['comment_text']]
ytr = Xtrain[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']]

Xts = Xtest[['comment_text']]

#### For now we want to have a glance at a baseline model, so we are not using nothing besides the comment_text with TF-IDF and a Logistic Regression with no tunning

- Not using the features created in 'EDA.ipynb'

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    use_idf=True,
    strip_accents='unicode',
    stop_words='english',
    min_df=30,
    ngram_range=(1, 1),
    max_features=10000)

feature_extractor = word_vectorizer

start = time.time()
print('extracting fetures')
train_features = feature_extractor.fit_transform(Xtr.comment_text)
test_features = feature_extractor.transform(Xts.comment_text)
print('fitted features time (seconds) : ',time.time() - start)

clf = LogisticRegression(class_weight=None) #'balanced'
score_df = pd.DataFrame(columns=ytr.columns)
scores = []
for col in ytr.columns:
    start2 = time.time()
    cv_score = np.mean(cross_val_score(clf,
                                       train_features,
                                       ytr[col],
                                       cv=3,
                                       scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for label {} is {}'.format(col, cv_score))

score_df = score_df.append(dict(zip(ytr.columns,scores)), ignore_index = True)

In [70]:
score_df

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.967367,0.985034,0.983077,0.982492,0.974352,0.973137
